<!--
authors: Matthew Wilson, Daniele Reda
created: 2020/01/14
last_updated: 2023/02/08
-->


## CPSC 533V: Assignment 2 - Tabular Q Learning and DQN (TBD)

---

#  Part 1 [54 pts] Tabular Q-Learning 

Tabular Q-learning is an RL algorithm for problems with discrete states and discrete actions. The algorithm is described in the class notes, which borrows the summary description from [Section 6.5](http://incompleteideas.net/book/RLbook2018.pdf#page=153) of Richard Sutton's RL book. In the tabular approach, the Q-value is represented as a lookup table. As discussed in class, Q-learning can further be extended to continuous states and discrete actions, leading to the [Atari DQN](https://arxiv.org/abs/1312.5602) / Deep Q-learning algorithm.  However, it is important and informative to first fully understand tabular Q-learning.

Informally, Q-learning works as follows: The goal is to learn the optimal Q-function: 
`Q(s,a)`, which is the *value* of being at state `s` and taking action `a`.  Q tells you how well you expect to do, on average, from here on out, given that you act optimally.  Once the Q function is learned, choosing an optimal action is as simple as looping over all possible actions and choosing the one with the highest Q (optimal action $a^* = \text{argmax}_a Q(s,a)$).  To learn Q, we initialize it arbitrarily and then iteratively refine it using the Bellman backup equation for Q functions, namely: 
$Q(s,a) \leftarrow Q(s,a) + \alpha [r + \gamma \text{max}_a Q(s', a) - Q(s,a)]$.
Here, $r$ is the reward associated with with the transition from state s to s', and $\alpha$ is a learning rate.

In the first part of assignment you will implement tabular Q-learning and apply it to CartPole -- an environment with a **continuous** state space.  To apply the tabular method, you will need to discretize the CartPole state space by dividing the state-space into bins.


**Goals:**
- to become familiar with python/numpy, as well as using an OpenAI Gym environment
- to understand tabular Q-learning, by implementing tabular Q-Learning for 
  a discretized version of a continuous-state environment, and experimenting with the implementation
- (optional) to develop further intuition regarding possible variations of the algorithm

## Introduction
Deep reinforcement learning has generated impressive results for board games ([Go][go], [Chess/Shogi][chess]), video games ([Atari][atari], [DOTA2][dota], [StarCraft II][scii]), [and][baoding] [robotic][rubix] [control][anymal] ([of][cassie] [course][mimic] ;)).  RL is beginning to work for an increasing range of tasks and capabilities.  At the same time, there are many [gaping holes][irpan] and [difficulties][amid] in applying these methods. Understanding deep RL is important if you wish to have a good grasp of the modern landscape of control methods.

These next several assignments are designed to get you started with deep reinforcement learning, to give you a more close and personal understanding of the methods, and to provide you with a good starting point from which you can branch out into topics of interest. You will implement basic versions of some of the important fundamental algorithms in this space, including Q-learning and policy gradient/search methods.

We will only have time to cover a subset of methods and ideas in this space.
If you want to dig deeper, we suggest following the links given on the course webpage.  Additionally we draw special attention to the [Sutton book](http://incompleteideas.net/book/RLbook2018.pdf) for RL fundamentals and in depth coverage, and OpenAI's [Spinning Up resources](https://spinningup.openai.com/en/latest/spinningup/rl_intro.html) for a concise intro to RL and deep RL concepts, as well as good comparisons and implementations of modern deep RL algorithms.


[atari]: https://arxiv.org/abs/1312.5602
[go]: https://deepmind.com/research/case-studies/alphago-the-story-so-far
[chess]:https://deepmind.com/blog/article/alphazero-shedding-new-light-grand-games-chess-shogi-and-go 
[dota]: https://openai.com/blog/openai-five/
[scii]: https://deepmind.com/blog/article/AlphaStar-Grandmaster-level-in-StarCraft-II-using-multi-agent-reinforcement-learning
[baoding]: https://bair.berkeley.edu/blog/2019/09/30/deep-dynamics/
[rubix]: https://openai.com/blog/solving-rubiks-cube/
[cassie]: https://www.cs.ubc.ca/~van/papers/2019-CORL-cassie/index.html
[mimic]: https://www.cs.ubc.ca/~van/papers/2018-TOG-deepMimic/index.html
[anymal]: https://arxiv.org/abs/1901.08652


[irpan]: https://www.alexirpan.com/2018/02/14/rl-hard.html
[amid]: http://amid.fish/reproducing-deep-rl



In [2]:
# # uncomment if necesary
!pip install numpy -i https://pypi.org/simple
# !pip install gym
# # OR:
!pip install gymnasium -i https://pypi.org/simple
import time
import itertools
import numpy as np
# import gym
import gymnasium as gym

---

## [12 pts] Explore the CartPole environment 

Your first task is to familiarize yourself with the OpenAI gym interface and the [CartPole environment](https://github.com/Farama-Foundation/Gymnasium/blob/main/gymnasium/envs/classic_control/cartpole.py)
by writing a simple hand-coded policy to try to solve it.  
To begin understanding OpenAI Gym environments, [read this first](https://gymnasium.farama.org/api/env/).) 
The gym interface is very popular and you will see many algorithm implementations and 
custom environments that support it.  You may even want to use the API in your course projects, 
to define a custom environment for a task you want to solve.

Note that there were several breaking changes introduced in the past few years to the gym API. Some reference algorithm implementations online might be using the old version:
- `obs = env.reset()` ->  `obs, info = env.reset()`
- `obs, reward, done, info = env.step(action)` to `obs, reward, terminated, truncated, info = env.step(action)`
- `env.render()` no longer accepts the `render_mode` parameter (e.g. human mode where the environment is rendered in a pop-up window, or rgb_array which allows headless conversion to images or videos)


Below is some example code that runs a simple random policy.  You are to:
- **run the code to see what it does**
- **write code that chooses an action based on the observation**.  You will need to learn about the gym API and to read the CartPole documentation to figure out what the `action` and `obs` vectors mean for this environment. 
Your hand-coded policy can be arbitrary, and it should ideally do better than the random policy.  There is no single correct answer. The goal is to become familiar with `env`s.
- **write code to print out the total reward gained by your policy in a single episode run**
- **answer the short-response questions below** (see the TODOs for all of this)

In [3]:
env = gym.make('CartPole-v1', render_mode="human")  # you can also try LunarLander-v2, but make sure to change it back
print('observation space:', env.observation_space)
print('action space:', env.action_space)

# To find out what the observations mean, read the CartPole documentation.
# Uncomment the lines below, or visit the source file: 
# https://github.com/Farama-Foundation/Gymnasium/blob/main/gymnasium/envs/classic_control/cartpole.py

# cartpole = env.unwrapped
# cartpole?

observation space: Box([-4.8               -inf -0.41887903        -inf], [4.8               inf 0.41887903        inf], (4,), float32)
action space: Discrete(2)


In [ ]:
# 1.1 [10pts]

# runs a single episode and render it.  try running this before editing anything
obs, info = env.reset()  # get first obs/state
sum_of_rewards = 0
while True:
    # TODO: replace this `action` with something that depends on `obs` 
    # action = env.action_space.sample()  # random action
    '''PD controller'''
    action = int(obs[2] * 0.5 + obs[3] * 0.5 - obs[0] * 0.5 > 0)
    obs, reward, terminated, truncated, info = env.step(action)
    sum_of_rewards += reward
    env.render()
    time.sleep(0.1)  # so it doesn't render too quickly
    if terminated | truncated: break
env.close()


# TODO: print out your total sum of rewards here
print('total sum of rewards:', sum_of_rewards) # XXX: consider the discount factor? (not necessary for this task)

To answer the questions below, look at the full [source code here](https://github.com/Farama-Foundation/Gymnasium/blob/main/gymnasium/envs/classic_control/cartpole.py) if you haven't already.

**1.2. [2pts] Briefly describe your policy.  What observation information does it use?  What score did you achieve (rough maximum and average)?  And how does it compare to the random policy?**

I simply used a PD controller to balance the pole. The policy uses the angle of the pole and the angular velocity of the pole to determine the action. Meanwhile, in order to keep the pole in the center, I also linearly add the position of the cart to the action, providing a negative feedback signal. The policy achieved a maximum score of 500.0 and an average score of 500.0 (within my 5 trials). The policy is much better than the random policy.

---

##  [12 pts] Discretize the env

Next, we need to discretize CartPole's continuous state space to work for tabular Q-learning.  While this is in part  a contrived usage of tabular methods, given the existence of other approaches that are designed to cope with continuous state-spaces, it is also interesting to consider whether tabular methods can be adapted more directly via discretization of the state into bins. Furthermore, tabular methods are simple, interpretabile, and can be proved to converge, and thus they still remain relevant.

Your task is to discretize the state/observation space so that it is compatible with tabular Q-learning.  To do this:
- **implement `obs_normalizer` to pass its test**
- **implement `get_bins` to pass its test**
- **then answer question 2.3**

[map]: https://arxiv.org/abs/1504.04909
[qd]: https://quality-diversity.github.io/

In [4]:
env = gym.make('CartPole-v1')

In [5]:
# 2.1 [5 pts for passing test_normed]
def obs_normalizer(obs):
    """Normalize the observations between 0 and 1
    
    If the observation has extremely large bounds, then clip to a reasonable range before normalizing; 
    (-2,2) should work.  (It is ok if the solution is specific to CartPole)
    
    Args:
        obs (np.ndarray): shape (4,) containing an observation from CartPole using the bound of the env
    Returns:
        normed (np.ndarray): shape (4,) where all elements are roughly uniformly mapped to the range [0, 1]
    
    """
    # HINT: check out env.observation_space.high, env.observation_space.low
    clip_value = 2.0
    low = env.observation_space.low
    high = env.observation_space.high
    res = np.zeros_like(obs)

    for i in range(len(obs)):
        # Check if the range for the observation is infinite
        if np.isinf(high[i] - low[i]):
            # Clip and map to [0, 1]
            res[i] = np.clip(obs[i], -clip_value, clip_value)
            res[i] = (res[i] + clip_value) / (2 * clip_value)
        else:
            # Normalize and clip to [0, 1]
            res[i] = (obs[i] - low[i]) / (high[i] - low[i])
            res[i] = np.clip(res[i], 0, 1)
    
    return res
    
    # TODO: implement this function
    raise NotImplementedError('TODO')

In [6]:
### TEST 2.1
def test_normed():
    obs, info = env.reset()
    while True:
        obs, _, terminated, truncated, _ =  env.step(env.action_space.sample())
        normed = obs_normalizer(obs) 
        assert np.all(normed >= 0.0) and np.all(normed <= 1.0), '{} are outside of (0,1)'.format(normed)
        if terminated | truncated: break
    env.close()
    print('Passed!')
test_normed()

Passed!


In [7]:
# 2.2 [5 pts for passing test_binned]
def get_bins(normed, num_bins):
    """Map normalized observations (0,1) to bin index values (0,num_bins-1)
    
    Args:
        normed (np.ndarray): shape (4,) output from obs_normalizer
        num_bins (int): how many bins to use
    Returns:
        binned (np.ndarray of type np.int32): shape (4,) where all elements are values in range [0,num_bins-1]
    
    """
    # print(normed)
    return (normed * (num_bins-1)).astype(np.int32)
    # TODO: implement this function
    raise NotImplementedError('TODO')

In [8]:
### TEST 2.2
obs, info = env.reset()

def test_binned(num_bins):
    normed = np.array([0.0, 0.2, 0.8, 1.0])
    binned = get_bins(normed, num_bins)
    assert np.all(binned >= 0) and np.all(binned < num_bins), '{} supposed to be between (0, {})'.format(binned, num_bins-1)
    assert binned.dtype == np.int32, "You should also make sure to cast your answer to int using arr.astype(np.int32)" 
    
test_binned(5)
test_binned(10)
test_binned(50)
print('Passed!')

Passed!


**2.3. [2 pts] If your state has 4 values and each is binned into N possible bins, how many bins are needed to represent all unique possible states?**



$N^4$ states are needed to represent all unique possible states.

---

## [20 pts] Solve the env 

Using the pseudocode below and the functions you implemented above, implement tabular Q-learning and use it to solve CartPole.

We provide setup code to initialize the Q-table and give examples of interfacing with it. Write the inner and outer loops to train your algorithm.  These training loops will be similar to those deep RL approaches, so get used to writing them!

The algorithm (excerpted from Section 6.5 of [Sutton's book](http://incompleteideas.net/book/RLbook2018.pdf)) is given below:

![Sutton RL](https://i.imgur.com/mdcWVRL.png)

in summary:
- **implement Q-learning using this pseudocode and the helper code**
- **answer the questions below**
- **run the suggested experiments and otherwise experiment with whatever interests you**

In [9]:
# setup (see last few lines for how to use the Q-table)

# hyper parameters. feel free to change these as desired and experiment with different values
num_bins = 20
alpha = 0.1
gamma = 0.99
log_n = 1000
# epsilon greedy
eps = 0.05  # usage: action = optimal if np.random.rand() > eps else random

obs, info = env.reset()

# Q-table initialized to zeros.  first 4 dims are state, last dim is for action (0,1) for left,right.
Q = np.zeros([num_bins]*len(obs)+[env.action_space.n])

# helper function to convert observation into a binned state so we can index into our Q-table
obs2bin = lambda obs: tuple(get_bins(obs_normalizer(obs), num_bins=num_bins))

s = obs2bin(obs)

print('Shape of Q Table: ', Q.shape) # you can imagine why tabular learning does not scale very well
print('Original obs {} --> binned {}'.format(obs, s))
print('Value of Q Table at that obs/state value', Q[s])

Shape of Q Table:  (20, 20, 20, 20, 2)
Original obs [ 0.03415893 -0.00279476  0.03935158  0.02917751] --> binned (np.int32(9), np.int32(9), np.int32(10), np.int32(9))
Value of Q Table at that obs/state value [0. 0.]


In [10]:
# 3.1 [20 pts]

# TODO: implement Q learning, following the pseudo-code above. 
#     - you can follow it almost exactly, but translating things for the gym api and our code used above
#     - make sure to use e-greedy, where e = random about 0.05 percent of the time
#     - make sure to do the S <-- S' step because it can be easy to forget
#     - every log_n steps, you should render your environment and
#       print out the average total episode rewards of the past log_n runs to monitor how your agent trains
#      (your implementation should be able to break at least +150 average reward value, and you can use that 
#       as a breaking condition.  It make take several minutes to run depending on your computer.)

# env = gym.make('CartPole-v1')

break_rewards = 150
reward_list = []
episode = 0

while True: # episode loop
    obs, info = env.reset()
    s = obs2bin(obs)
    sum_of_rewards = 0

    while True: # step loop
        # get action using epsilon greedy
        if np.random.rand() > eps:
            a = np.argmax(Q[s])
        else:
            a = env.action_space.sample()
        # take action
        obs, reward, terminated, truncated, info = env.step(a)
        # update
        s_prime = obs2bin(obs)
        Q[s][a] = Q[s][a] + alpha * (reward + gamma * np.max(Q[s_prime]) - Q[s][a])
        s = s_prime
        sum_of_rewards += reward
        # exit condition
        if terminated | truncated: break

    # log reward
    reward_list.append(sum_of_rewards)
    avg_reward = np.mean(reward_list[-log_n:])
    # show the environment every log_n episodes
    if episode % log_n == 0 and episode > 0:
        print('Average reward between episodes {} and {}: {}'.format(episode-log_n, episode, avg_reward))
        # XXX: uncomment to show the environment
        # env.render()
        # time.sleep(0.1)
    # exit condition
    if avg_reward > break_rewards: break
    # increment episode
    episode += 1
        
        

Average reward between episodes 0 and 1000: 9.799
Average reward between episodes 1000 and 2000: 10.472
Average reward between episodes 2000 and 3000: 14.679
Average reward between episodes 3000 and 4000: 21.77
Average reward between episodes 4000 and 5000: 37.354
Average reward between episodes 5000 and 6000: 54.161
Average reward between episodes 6000 and 7000: 68.106
Average reward between episodes 7000 and 8000: 74.821
Average reward between episodes 8000 and 9000: 103.933
Average reward between episodes 9000 and 10000: 142.315


In [12]:
test_env = gym.make('CartPole-v1', render_mode="human")
for _ in range(10):
    obs, info = test_env.reset()
    sum_of_rewards = 0
    while True:
        s = obs2bin(obs)
        a = np.argmax(Q[s])
        obs, reward, terminated, truncated, info = test_env.step(a)
        sum_of_rewards += reward
        if terminated | truncated: break
    print('test episode reward:', sum_of_rewards)

2024-10-18 17:41:34.784 python[36599:1477436] +[IMKClient subclass]: chose IMKClient_Legacy
2024-10-18 17:41:34.784 python[36599:1477436] +[IMKInputSession subclass]: chose IMKInputSession_Legacy


test episode reward: 270.0
test episode reward: 500.0
test episode reward: 252.0
test episode reward: 500.0
test episode reward: 151.0
test episode reward: 361.0
test episode reward: 225.0
test episode reward: 194.0
test episode reward: 309.0
test episode reward: 500.0


## [10 pts] Experiments

Given a working algorithm, you will run a few experiments.  Either make a copy of your code above to modify, or make the modifications in a way that they can be commented out or switched between (with boolean flag if statements).

In [17]:
from torch.utils.tensorboard import SummaryWriter

In [15]:
def run_experiment(num_bins=20, alpha=0.1, gamma=0.99, log_n=1000, eps=0.05, name='experiment'):
    obs2bin = lambda obs: tuple(get_bins(obs_normalizer(obs), num_bins=num_bins))
    env = gym.make('CartPole-v1')
    obs, info = env.reset()
    Q = np.zeros([num_bins]*len(obs)+[env.action_space.n])
    break_rewards = 150
    reward_list = []
    episode = 0
    max_episodes = 10000
    writer = SummaryWriter(log_dir=f'{name}/experiment_num_bins={num_bins}_alpha={alpha}_gamma={gamma}_log_n={log_n}_eps={eps}')
    
    while True: # episode loop
        obs, info = env.reset()
        s = obs2bin(obs)
        sum_of_rewards = 0

        while True: # step loop
            # get action using epsilon greedy
            if np.random.rand() > eps:
                a = np.argmax(Q[s])
            else:
                a = env.action_space.sample()
            # take action
            obs, reward, terminated, truncated, info = env.step(a)
            # update
            s_prime = obs2bin(obs)
            Q[s][a] = Q[s][a] + alpha * (reward + gamma * np.max(Q[s_prime]) - Q[s][a])
            s = s_prime
            sum_of_rewards += reward
            # exit condition
            if terminated | truncated: break

        # log reward
        reward_list.append(sum_of_rewards)
        writer.add_scalar('reward', sum_of_rewards, episode)

        # Show the environment every log_n episodes
        avg_reward = np.mean(reward_list[-log_n:])
        # exit condition
        if avg_reward > break_rewards or episode >= max_episodes: break
        # increment episode
        episode += 1
    
    writer.close()
    return Q, reward_list
    

**4.2. [5 pts] $\epsilon$-greedy.**  How sensitive are the results to the value of $\epsilon$?   First, write down your prediction of what would happen if $\epsilon$ is set to various values, including for example [0, 0.05, 0.25, 0.5].

For a small $\epsilon$ like $0$, the agent will exploit the policy more. This will lead to the agent not exploring the environment enough and result in not learning the optimal policy. For a large $\epsilon$ like $0.5$, the agent will explore the environment more. This will lead to the agent not exploiting the policy enough and add too much noise to the training process, resulting in a slow convergence. For $\epsilon$ like $0.05, 0.25$, the agent will balance between exploration and exploitation.

Now run the experiment and observe the impact on the algorithm.  Report the results below.

In [25]:
eps_list = [0, 0.05, 0.25, 0.5]
for eps in eps_list:
    print('--- Running experiment with epsilon {} ---'.format(eps))
    Q, reward_list = run_experiment(eps=eps, name='eps_experiment')

--- Running experiment with epsilon 0 ---
--- Running experiment with epsilon 0.05 ---
--- Running experiment with epsilon 0.25 ---
--- Running experiment with epsilon 0.5 ---


In [ ]:
%load_ext tensorboard
%tensorboard --logdir runs

Same as my prediction :)
![eps experiment](./result/Capture-2024-10-18-124926.png)

**4.3. [5 pts] Design your own experiment.** Design a modification that you think would either increase or reduce performance.  A simple example (which you can use) is initializing the Q-table differently, and thinking about how this might alter performance. Write down your idea, what you think might happen, and why.

I tried to change different number of bins for the discretized state space. I think that the more bins we have, the more accurate the Q-table will be. However, the more bins we have, the more states we need to explore, which will slow down the training process.

Run the experiment and report the results.

In [20]:
num_bin_list = [8, 16, 32, 64, 128, 256]
for num_bins in num_bin_list:
    print('--- Running experiment with num_bins =', num_bins)
    Q, reward_list = run_experiment(num_bins=num_bins, name='num_bins_experiment')

--- Running experiment with num_bins = 8
--- Running experiment with num_bins = 16
--- Running experiment with num_bins = 32
--- Running experiment with num_bins = 64
--- Running experiment with num_bins = 128
--- Running experiment with num_bins = 256


![bins experiment](./result/Capture-2024-10-18-234049.png)
As show in the figure, only 16 bins and 32 bins can solve the environment. The more bins we have, the more states we need to explore, which will slow down the training process. Besides, if the bins are too small (8 bins), the agent will not be able to explore the environment enough and result in not learning the optimal policy.

---

## A. Extensions (fully optional, will not be graded, if you have time after Part 2)

- plots your learning curve, using e.g., matplotlib 
- visualize the Q-table to see which values are being updated and not
- design a better binning strategy that uses fewer bins for a better-performing policy
- extend this approach to work on different environments (e.g., LunarLander-v2)
- extend this approach to work on environments with continuous actions, by using a fixed set of discrete samples of the action space.  e.g., for Pendulum-v0
- implement a simple deep learning version of this.  we will see next part that DQN uses some tricks to make the neural network training more stable.  Experiment directly with simply replacing the Q-table with a Q-Network and train the Q-Network using gradient descent with `loss = (targets - Q(s,a))**2`, where `targets = stop_grad(R + gamma * maxa(Q(s,a))`).

# Part 2 [60 pts] Behavioral Cloning and Deep Q Learning

---
The second part of assignment will help you transition from tabular approaches to deep neural network approaches. You will implement the [Atari DQN / Deep Q-Learning](https://arxiv.org/abs/1312.5602) algorithm, which arguably kicked off the modern Deep Reinforcement Learning craze.

In this part we will use PyTorch as our deep learning framework.  To familiarize yourself with PyTorch, your first task is to use a behavior cloning (BC) approach to learn a policy.  Behavior cloning is a supervised learning method in which there exists a dataset of expert demonstrations (state-action pairs) and the goal is to learn a policy $\pi$ that mimics this expert.  At any given state, your policy should choose the same action the export would.

Since BC avoids the need to collect data from the policy you are trying to learn, it is relatively simple. 
This makes it a nice stepping stone for implementing DQN. Furthermore, BC is relevant to modern approaches---for example its use as an initialization for systems like [AlphaGo][go] and [AlphaStar][star], which then use RL to further adapte the BC result.  

<!--

I feel like this might be better suited to going lower in the document:

Unfortunately, in many tasks it is impossible to collect good expert demonstrations, making

it's not always possible to have good expert demonstrations for a task in an environemnt and this is where reinforcement learning comes handy. Through the reward signal retrieved by interacting with the environment, the agent learns by itself what is a good policy and can learn to outperform the experts.

-->

Goals:
- Famliarize yourself with PyTorch and its API including models, datasets, dataloaders
- Implement a supervised learning approach (behavioral cloning) to learn a policy.
- Implement the DQN objective and learn a policy through environment interaction.

[go]:  https://deepmind.com/research/case-studies/alphago-the-story-so-far
[star]: https://deepmind.com/blog/article/alphastar-mastering-real-time-strategy-game-starcraft-ii

## Submission information

- Complete by editing and executing the associated Python files.
- Copy and paste the code and the terminal output requested in the predefined cells on this Jupyter notebook.
- When done, upload the completed Jupyter notebook (ipynb file) on canvas.

## Preliminaries

### PyTorch

If you have never used PyTorch before, we recommend you follow this [60 Minutes Blitz][blitz] tutorial from the official website. It should give you enough context to be able to complete the assignment.


**If you have issues, post questions to Piazza**

### Installation

To install all required python packages:

```
python3 -m pip install -r requirements.txt
```

### Debugging


You can include:  `import ipdb; ipdb.set_trace()` in your code and it will drop you to that point in the code, where you can interact with variables and test out expressions.  We recommend this as an effective method to debug the algorithms.


[blitz]: https://pytorch.org/tutorials/beginner/deep_learning_60min_blitz.html

## 1. [36 pts] Behavioral Cloning

Behavioral Cloning is a type of supervised learning in which you are given a dataset of expert demonstrations tuple $(s, a)$ and the goal is to learn a policy function $\hat a = \pi(s)$, such that $\hat a = a$.

The optimization objective is $\min_\theta D(\pi(s), a)$ where $\theta$ are the parameters the policy $\pi$, in our case the weights of a neural network, and where $D$ represents some difference between the actions.

---

Before starting, we suggest reading through the provided files.

For Behavioral Cloning, the important files to understand are: `model.py`, `dataset.py` and `bc.py`.

- The file `model.py` has the skeleton for the model (which you will have to complete in the following questions),

- The file `dataset.py` has the skeleton for the dataset the model is being trained with,

- and, `bc.py` will have all the structure for training the model with the dataset.


### [10 pts] 1.1 Dataset

We provide a pickle file with pre-collected expert demonstrations on CartPole from which to learn the policy $\pi$. The data has been collected from an expert policy on the environment, with the addition of a small amount of gaussian noise to the actions.

The pickle file contains a list of tuples of states and actions in `numpy` in the following way:

```
[(state s, action a), (state s, action a), (state s, action a), ...]
```

In the `dataset.py` file, we provide skeleton code for creating a custom dataset. The provided code shows how to load the file.

Your goal is to overwrite the `__getitem__` function in order to return a dictionary of tensors of the correct type.

Hint: Look in the `bc.py` file to understand how the dataset is used.

Answer the following questions:

**[6 pts]** Insert your code in the placeholder below.

In [13]:
# PLACEHOLDER TO INSERT YOUR __getitem__ method here

def __getitem__(self, index):
    item = self.data[index]
    # TODO YOUR CODE HERE
    return {
        "state": torch.tensor(item[0], dtype=torch.float32),
        "action": torch.tensor(item[1], dtype=torch.long),
    }

**[2 pt]** How big is the dataset provided?

99660 samples are provided in the dataset.

**[2 pts]** What is the dimensionality of $s$ and what range does each dimension of $s$ span?  I.e., how much of the state space does the expert data cover? What are the dimensionalities and ranges of the action $a$ in the dataset (how much of the action space does the expert data cover)?

The dimensionality of $s$ is 4. 

The range of each dimension of $s$ is $[-0.723, 2.399]$, $[-0.433, 1.847]$, $[-0.050, 0.146]$, $[-0.381, 0.471]$. 

The dimensionality of $a$ is 1. 

The range of $a$ is $[0, 1]$.



### [5 pts] 1.2 Environment

Recall the state and action space of CartPole, from the previous assignment.

Considering the full state and action spaces, do you think the provided expert dataset has good coverage?  Why or why not? How might this impact the performance of our cloned policy?

This expert does not have good coverage of the state space. Because the expert dataset only covers the center part of the state space (maybe due to the expert is so good at balancing the pole), the cloned policy will not be able to handle the out-of-distribution states well. This may lead to the cloned policy not being able to balance the pole as well as the expert at certain extreme states.

### [14 pts] 1.3 Model

The file `model.py` provides skeleton code for the model. Your goal is to create the architecture of the network by adding layers that map the input to output.

You will need to update the `__init__` method and the `forward` method.

The `select_action` method has already been written for you.  This should be used when running the policy in the environment, while the `forward` function should be used at training time.

- **[10 pts]** Insert your code in the placeholder below.

In [ ]:
# PLACEHOLDER TO INSERT YOUR MyModel class here

class MyModel(nn.Module):
    def __init__(self, state_size, action_size):
        super().__init__()
        # TODO YOUR CODE HERE FOR INITIALIZING THE MODEL
        # Guidelines for network size: start with 2 hidden layers and maximum 32 neurons per layer
        # feel free to explore different sizes
        self.fc1 = nn.Linear(state_size, 16)
        self.fc2 = nn.Linear(16, 16)
        self.fc3 = nn.Linear(16, action_size)

    def forward(self, x):
        # TODO YOUR CODE HERE FOR THE FORWARD PASS
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

        raise NotImplementedError()

    def select_action(self, state):
        self.eval()
        x = self.forward(state)
        self.train()
        return x.max(1)[1].view(1, 1).to(torch.long)

Answer the following questions:

- **[2 pts]** What is the dimension and meaning of the input of the network

The dimension of the input of the network is 4. The input is the state of the CartPole environment.

- **[2 pts]** Similarly, describe the output.

The dimension of the output of the network is 2. The output is the action of the CartPole environment. If the first node is larger than the second node, the action is 0. Otherwise, the action is 1.


### [7 pts] 1.4 Training

The file `bc.py` is the entry point for training your behavioral cloning model. The skeleton and the main components are already there.

The missing parts for you to do are:

- Initializing the model
- Choosing a loss function
- Choosing an optimizer
- Playing with hyperparameters to train your model.

- **[5 pts]** Insert your code in the placeholder below.

In [ ]:
# PLACEHOLDER FOR YOUR CODE HER
# HOW DID YOU INITIALIZE YOUR MODEL, OPTIMIZER AND LOSS FUNCTIONS? PASTE HERE YOUR FINAL CODE
# NOTE: YOU CAN KEEP THE FOLLOWING LINES COMMENTED OUT, AS RUNNING THIS CELL WILL PROBABLY RESULT IN ERRORS

model = MyModel(
    state_size=env.observation_space.shape[0],
    action_size=env.action_space.n,
).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
loss_function = torch.nn.CrossEntropyLoss()

You can run your code by doing:

```
python3 bc.py
```

**During all of this assignment, the code in `eval_policy.py` will be your best friend.** At any time, you can test your model by giving as argument the path to the model weights and the environment name using the following command:

```
python3 eval_policy.py --model-path /path/to/model/weights --env ENV_NAME
````

In [ ]:
## PASTE YOUR TERMINAL OUTPUT HERE
# NOTE: TO HAVE LESS LINES PRINTED, YOU CAN SET THE VARIABLE PRINT_INTERVAL TO 5 or 10
[epoch    1/20] [iter       0] [loss 0.70834]
[epoch    2/20] [iter     500] [loss 0.15327]
[Test on environment] [epoch 2/20] [score 282.40]
[epoch    3/20] [iter    1000] [loss 0.07436]
[epoch    4/20] [iter    1500] [loss 0.03162]
[Test on environment] [epoch 4/20] [score 244.70]
[epoch    6/20] [iter    2000] [loss 0.04218]
[Test on environment] [epoch 6/20] [score 258.60]
[epoch    7/20] [iter    2500] [loss 0.06094]
[epoch    8/20] [iter    3000] [loss 0.01773]
[Test on environment] [epoch 8/20] [score 254.80]
[epoch    9/20] [iter    3500] [loss 0.01822]
[Test on environment] [epoch 10/20] [score 252.70]
[epoch   11/20] [iter    4000] [loss 0.02184]
[epoch   12/20] [iter    4500] [loss 0.05812]
[Test on environment] [epoch 12/20] [score 238.70]
[epoch   13/20] [iter    5000] [loss 0.01225]
[Test on environment] [epoch 14/20] [score 250.60]
[epoch   15/20] [iter    5500] [loss 0.04915]
[epoch   16/20] [iter    6000] [loss 0.02374]
[Test on environment] [epoch 16/20] [score 284.80]
[epoch   17/20] [iter    6500] [loss 0.00530]
[epoch   18/20] [iter    7000] [loss 0.02306]
[Test on environment] [epoch 18/20] [score 251.50]
[epoch   20/20] [iter    7500] [loss 0.01703]
[Test on environment] [epoch 20/20] [score 286.40]

**[2 pts]** Did you manage to learn a good policy? How consistent is the reward you are getting?

It's definitely not a good policy. The reward is cosistently around 200, but can't reach higher. It always fails to balance the pole for a long time.

## 2. [24 pts] Deep Q Learning

There are two main issues with the behavior cloning approach.

- First, we are not always lucky enough to have access to a dataset of expert demonstrations.
- Second, replicating an expert policy suffers from compounding error. The policy $\pi$ only sees these "perfect" examples and has no knowledge on how to recover from states not visited by the expert. For this reason, as soon as it is presented with a state that is off the expert trajectory, it will perform poorly and will continue to deviate from a good trajectory without the possibility of recovering from errors.

---
The second task consists in solving the environment from scratch, using RL, and most specifically the DQN algorithm, to learn a policy $\pi$.

For this task, familiarize yourself with the file `dqn.py`. We are going to re-use the file `model.py` for the model you created in the previous task.

Your task is very similar to the one in the previous assignment, to implement the Q-learning algorithm, but in this version, our Q-function is approximated with a neural network.

The algorithm (excerpted from [Atari DQN paper](https://arxiv.org/abs/1312.5602)) is given below:

![DQN algorithm](https://i.imgur.com/Mh4Uxta.png)

### 2.0 [2 pts] Think about your model...



In DQN, we are using the same model as in task 1 for behavioral cloning. In both tasks the model receives as input the state and in both tasks the model outputs something that has the same dimensionality as the number of actions. These two outputs, though, represent very different things. What is each one representing?

In behavioral cloning, the output is treated as classification scores that can be passed through a softmax to get a probability distribution over actions. This is used to select the most likely action (based on expert behavior) for a given state.

In DQN, the model outputs a vector where each element corresponds to the Q-value (expected future reward) of taking a particular action in the given state.

### 2.1 [10 pts] Update your Q-function

Complete the `optimize_model` function. This function receives as input a `state`, an `action`, the `next_state`, the `reward` and `done` representing the tuple $(s_t, a_t, s_{t+1}, r_t, done_t)$. Your task is to update your Q-function as shown in the [Atari DQN paper](https://arxiv.org/abs/1312.5602) environment. For now don't be concerned with the experience replay buffer. We'll get to that later.

![Loss function](https://i.imgur.com/tpTsV8m.png)

Insert your code in the placeholder below.

In [ ]:
## PLACEHOLDER TO INSERT YOUR optimize_model function here:

def optimize_model(state, action, next_state, reward, done):
    # TODO given a tuple (s_t, a_t, s_{t+1}, r_t, done_t) update your model weights
    state = torch.tensor(state, device=device, dtype=torch.float32).unsqueeze(0)
    next_state = torch.tensor(next_state, device=device, dtype=torch.float32).unsqueeze(0)
    y_i = reward + (1 - done) * GAMMA * target(next_state).max(1)[0].detach()
    loss = F.mse_loss(y_i.unsqueeze(1), model(state).gather(1, action))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

### 2.2 [5 pts] $\epsilon$-greedy strategy

You will need a strategy to explore your environment. The standard strategy is to use $\epsilon$-greedy. Implement it in the `choose_action` function template.

Insert your code in the placeholder below.

In [ ]:
## PLACEHOLDER TO INSERT YOUR choose_action function here:

def choose_action(state, test_mode=False):
    if not test_mode and random.random() < EPS_EXPLORATION:
        return torch.tensor(env.action_space.sample(), device=device).view(1, 1)
    else:
        state = torch.tensor(state, device=device, dtype=torch.float32).unsqueeze(0)
        return model.select_action(state)

### 2.3 [2 pts] Train your model

Try to train a model in this way.

You can run your code by doing:

```
python3 dqn.py
```

How many episodes does it take to learn (ie. reach a good reward)?

Through a lot of trials, it takes around 1000-2000 episodes to learn.

In [ ]:
## PASTE YOUR TERMINAL OUTPUT HERE
# NOTE: TO HAVE LESS LINES PRINTED, YOU CAN SET THE VARIABLE PRINT_INTERVAL TO 5 or 10
[Episode   50/1000] [Steps    8] [reward 9.0]
[Episode  100/1000] [Steps   43] [reward 44.0]
----------
saving model.
[TEST Episode 100] [Average Reward 38.4]
----------
[Episode  150/1000] [Steps    9] [reward 10.0]
[Episode  200/1000] [Steps   11] [reward 12.0]
----------
[TEST Episode 200] [Average Reward 9.5]
----------
[Episode  250/1000] [Steps    9] [reward 10.0]
[Episode  300/1000] [Steps   11] [reward 12.0]
----------
[TEST Episode 300] [Average Reward 10.7]
----------
[Episode  350/1000] [Steps   64] [reward 65.0]
[Episode  400/1000] [Steps    9] [reward 10.0]
----------
[TEST Episode 400] [Average Reward 12.2]
----------
[Episode  450/1000] [Steps   11] [reward 12.0]
[Episode  500/1000] [Steps    9] [reward 10.0]
----------
[TEST Episode 500] [Average Reward 16.5]
----------
[Episode  550/1000] [Steps    9] [reward 10.0]
[Episode  600/1000] [Steps   15] [reward 16.0]
----------
saving model.
[TEST Episode 600] [Average Reward 43.9]
----------
[Episode  650/1000] [Steps  221] [reward 222.0]
[Episode  700/1000] [Steps   31] [reward 32.0]
----------
saving model.
[TEST Episode 700] [Average Reward 52.7]
----------
[Episode  750/1000] [Steps  109] [reward 110.0]
[Episode  800/1000] [Steps    8] [reward 9.0]
----------
[TEST Episode 800] [Average Reward 9.3]
----------
[Episode  850/1000] [Steps   12] [reward 13.0]
[Episode  900/1000] [Steps  115] [reward 116.0]
----------
saving model.
[TEST Episode 900] [Average Reward 61.7]
----------
[Episode  950/1000] [Steps  117] [reward 118.0]
[Episode 1000/1000] [Steps   51] [reward 52.0]
----------
saving model.
[TEST Episode 1000] [Average Reward 326.1]
----------

### 2.4 [5 pts] Add the Experience Replay Buffer

If you read the DQN paper (and as you can see from the algorithm picture above), the authors make use of an experience replay buffer to learn faster. We provide an implementation in the file `replay_buffer.py`. Update the `train_reinforcement_learning` code to push a tuple to the replay buffer and to sample a batch for the `optimize_model` function.

In [ ]:
## PASTE YOUR TERMINAL OUTPUT HERE
# NOTE: TO HAVE LESS LINES PRINTED, YOU CAN SET THE VARIABLE PRINT_INTERVAL TO 5 or 10
Episode   10/4000] [Steps    8] [reward 9.0]
[Episode   20/4000] [Steps    8] [reward 9.0]
[Episode   30/4000] [Steps   13] [reward 14.0]
[Episode   40/4000] [Steps   11] [reward 12.0]
[Episode   50/4000] [Steps   10] [reward 11.0]
[Episode   60/4000] [Steps   10] [reward 11.0]
[Episode   70/4000] [Steps    8] [reward 9.0]
[Episode   80/4000] [Steps    8] [reward 9.0]
[Episode   90/4000] [Steps    8] [reward 9.0]
[Episode  100/4000] [Steps    8] [reward 9.0]
----------
saving model.
[TEST Episode 100] [Average Reward 9.1]
----------
[Episode  110/4000] [Steps    7] [reward 8.0]
[Episode  120/4000] [Steps    7] [reward 8.0]
[Episode  130/4000] [Steps   15] [reward 16.0]
[Episode  140/4000] [Steps   10] [reward 11.0]
[Episode  150/4000] [Steps   11] [reward 12.0]
[Episode  160/4000] [Steps   10] [reward 11.0]
[Episode  170/4000] [Steps   10] [reward 11.0]
[Episode  180/4000] [Steps    9] [reward 10.0]
[Episode  190/4000] [Steps    9] [reward 10.0]
[Episode  200/4000] [Steps   19] [reward 20.0]
----------
saving model.
[TEST Episode 200] [Average Reward 38.6]
----------
[Episode  210/4000] [Steps   31] [reward 32.0]
[Episode  220/4000] [Steps  104] [reward 105.0]
[Episode  230/4000] [Steps   72] [reward 73.0]
[Episode  240/4000] [Steps  150] [reward 151.0]
[Episode  250/4000] [Steps  206] [reward 207.0]
[Episode  260/4000] [Steps  260] [reward 261.0]
[Episode  270/4000] [Steps  353] [reward 354.0]
[Episode  280/4000] [Steps  284] [reward 285.0]
[Episode  290/4000] [Steps  228] [reward 229.0]
[Episode  300/4000] [Steps  223] [reward 224.0]
----------
saving model.
[TEST Episode 300] [Average Reward 227.4]

# it is too slow to run the full 4000 episodes, so I will stop it here, I believe the following reward will be better

How does the replay buffer improve performance?

The data-efficiency is improved by the replay buffer significantly. The episodes needed to learn to the same level as without replay buffer are reduced a lot. 

This is because the replay buffer allows the agent to learn from past experiences multiple times, which helps to stabilize the learning process and reduce the variance of the updates.

However, the time for each episode is increased due to the extra computation of sampling from the replay buffer.

## 3. Extra (fully optional)

Ideas to experiment with:

- Is $\epsilon$-greedy strategy the best strategy available? Experiment with other strategies.
- Make use of the model you have trained in the behavioral cloning part and fine-tune it with RL. How does that affect performance?
- You are perhaps bored with `CartPole-v1` by now. Another environment we suggest trying is `LunarLander-v2`. It will be harder to learn but with experimentation, you will find the correct optimizations for success. Piazza is also your friend :)
- What about learning from images? This requires more work because you have to extract the image from the environment. How much more challenging might you expect the learning to be in this case?
- An improvement over DQN is DoubleDQN. Experiment with this to see how much of an impact it makes.



In [ ]:
# YOU CAN USE THIS CODEBLOCK AND ADD ANY BLOCK BELOW AS YOU NEED
# TO SHOW US THE IDEAS AND EXTRA EXPERIMENTS YOU RUN.
# HAVE FUN!